#**Certificate Automation**

Please fill in details like the **path of template certificate**, **path of excel file with email addresses**, **path of output folder** and according to the template certificate change the **font size, font thickness, font type and the x and y coordinates.**
 

In [ ]:
import cv2 as cv
import openpyxl
def certficategen():

  template_path = r'<your certificate template path>'
  addresses_path = r'<path of excel sheet containing name,email address columns>'
  output_path = r'<path of folder in which you want to save outputs>'

  # to save a list email addresses that caused exceptions while sending
  failed_recipient_list = []  

  # Setting the font size and colour - CHANGE ACCORDING TO CERTIFICATE TEMPLATES 
  font_size = 2
  font_color = (70, 70, 70)  #grey color

  # Coordinates on the certificate where name is printed 
  # (set according to your own template)
  coordinate_y_adjustment = 60
  coordinate_x_adjustment = 5

  # loading the trial.xlsx workbook and seleting the active sheet
  obj = openpyxl.load_workbook(addresses_path)
  sheet = obj.active

  # run the for loop through the length of the excel sheet 
  # remember : Excel indexing starts from 1 in this case
  for i in range(1, 30):
    # Selected the ith row of the excel sheet
    # cell is stored in the variable name
    cell_name = sheet.cell(row=i, column=1)
    cell_email = sheet.cell(row=i, column=2)
    name = cell_name.value
    mail = cell_email.value

    # loading the certificate template
    img = cv.imread(template_path)

    # choosing any font from the opencv fonts set
    font = cv.FONT_HERSHEY_COMPLEX
    text_size = cv.getTextSize(name, font, font_size, 4)[0]

    # get the (x,y) coordinates where the
    # name is to written on the template
    # The function cv.putText accepts only
    # integer arguments so convert it into 'int'.
    text_x = (img.shape[1] - text_size[0]) / 2 + coordinate_x_adjustment
    text_y = (img.shape[0] + text_size[1]) / 2 - coordinate_y_adjustment

    # putText function accepts int positions, thus converting to int here
    text_x = int(text_x)
    text_y = int(text_y)

    cv.putText(img, name, (text_x, text_y), font, font_size, font_color, 4)

    # Certificates will be saved with the name of the user in the output folder
    certi_path = output_path + '/' + name + '.jpg'

    # Writing/Saving the certificate
    cv.imwrite(certi_path, img)

    # If you wish to convert the certificate to pdf : 
    '''
    from PIL import Image
    image_jpg = Image.open(output_path + name +'.jpg')
    image_pdf = image_jpg.convert('RGB')
    image_pdf.save(output_path + name +'.pdf')
    '''
    
    if mail == 'No mail' or mail == '':
      print("No mail for name:", name)
      failed_recipient_list.append((name, mail))
      continue
    else:
      emailsending(name, mail, failed_recipient_list)
    print('Done!')

  print("Number of failed emails:", len(failed_recipient_list))
  print("List of failed emails:",failed_recipient_list)
  

def emailsending(recipient_name, email_receiver, failed_recipient_list):
  # Email sending
  from email.mime.application import MIMEApplication
  from email.mime.multipart import MIMEMultipart
  from email.mime.text import MIMEText
  from os.path import basename
  import smtplib

  email_sender = 'email@gmail.com'
  email_password = '<To generate this, enable 2-step verification and then set an app password which is added here'
  # 16 char password after turning 2-step verification
  # Go to Google account -> 2-Step verification -> follow steps and add phone number etc -> 
  # Create App password -> Enter password here (and save it else where!!!)

  subject = "<email subject>"
  body = '''Greetings!

Change Is Us thanks you for joining us at the Beach Cleanup held on <date>.

Our Mission is to make each citizen of India as well as the World aware of the climate change and take action against it and you helped us do just that by being present. Your dedication to making a change is beautiful and appreciated.

We work in close coordination and collaboration with the local authorities who have been really supportive. We wish to continue this initiative and do our bit to contribute to the country. It’s time the people of Mumbai understand a therapeutic beach clean up at Chowpatty is a brilliant way to spend one’s weekend.
Apart from spreading awareness in public, we also use social media (@change.is.us) to spread awareness about climate change, plastic pollution and the importance of the environment.

If you liked this event, Change Is Us promises to come up with such fun-filled events that allow you to be one with nature while you network with others too. Do join our upcoming beach clean ups, donation drives and other on-field engagement activities by checking out our website www.changeisus.in !

Do reach out to us if you have any questions!

Warm Regards,
Team Change Is Us.'''

  em = MIMEMultipart()
  em['From'] = email_sender
  em['To'] = email_receiver
  em['Subject'] = subject
  em.attach(MIMEText(body, 'plain'))

  filename = '<output_path_here>' + recipient_name + '.jpg'
  with open(filename, 'rb') as f:
    part = MIMEApplication(f.read(), Name=basename(filename))
    part['Content-Disposition'] = 'attachment; filename="{}"'.format(basename(filename))
  em.attach(part)

  session = smtplib.SMTP('smtp.gmail.com', 587)
  session.starttls()  # enable security
  session.login(email_sender, email_password)
  text = em.as_string()

  try:
    session.sendmail(email_sender, email_receiver, text)
    print('Email sent to ' + email_receiver + ' (Name: ' + recipient_name + ')\n')
  except SMTPException:
    print("Error sending mail to " + email_receiver + ' (Name: ' + recipient_name + ')\n')
    failed_recipient_list.append([recipient_name, email_receiver])
  session.quit()

# Calling certificategen() function 
certficategen()